## CS 5242 Project Bonus, NUS 2020 Semester2

**Group Name: Team 2333**

**Group members:**
- Li Fengzi A0206468Y 
- Xiuping Hua A0206514N
- Shi Haohui A0206537E
- Jonathan Simon Wagner A0152784X

**Project address: https://www.kaggle.com/c/cs5242project/data**

- Data: I3D features, dimension is D=400
- Training videos: P16-P54
- Test videos: P03-P15
- Total 1712 videos
- Ground truth: frame-wise sub-action labels for each video. Dim = Ni*1 (total num of frames in video i, including 'SIL') 
- Splits:4 splits, split1 - test, train on other splits
- 48 different sub-actions including SIL (48 classes)
- 10 breakfast activities
- Each video contain Ni frames, various sub-actions, so various segments, thus various segment labels
- data_feat: I3D features for all training videos, dimension = Ni * D, Ni-frames, D=400
- data_labels: a list of all training labels, a list variable of length = the number of training segment for this video clip, i.e. segment labels, not the frame labels

- load data: template_code.py

### 1. Load libraries

In [0]:
%tensorflow_version 1.x
import tensorflow

TensorFlow 1.x selected.


In [0]:
# Load library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# mounted at google drive
from google.colab import drive
drive.mount('/content/gdrive')
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'CS5242'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cat '/content/gdrive/My Drive/CS5242/read_datasetBreakfast.py'

In [0]:
# Update path to import from drive
import sys
sys.path.append('/content/gdrive/My Drive/CS5242')

### 2. Load data

In [0]:
from read_datasetBreakfast import load_data, read_mapping_dict
import os

COMP_PATH = 'content/gdrive/My Drive/CS5242/'

''' 
training to load train set
test to load test set
'''

train_split =  os.path.join(base_dir, 'splits/train.split1.bundle') #Train Split
test_split  =  os.path.join(base_dir, 'splits/test.split1.bundle') #Test Split
GT_folder   =  os.path.join(base_dir, 'groundTruth/groundTruth/') #Ground Truth Labels for each training video 
DATA_folder =  os.path.join(base_dir, 'data/') #Frame I3D features for all videos
mapping_loc =  os.path.join(base_dir, 'splits/mapping_bf.txt')

actions_dict = read_mapping_dict(mapping_loc)

data_feat, data_labels = load_data( train_split, actions_dict, GT_folder, DATA_folder, datatype = 'training') #Get features and labels
data_feat_test = load_data( test_split, actions_dict, GT_folder, DATA_folder, datatype = 'test') #Get features only

'''
Write Code Below
Pointers
Need to load the segments.txt file for segments for test videos 
Output the CSV in correct format as shown in Evaluation Section
Id corresponds to the segments in order. 
Example - 30-150 = Id 0
          150-428 = Id 1
          428-575 = Id 2
Category is the Class of the Predicted Action
'''

Finish Load the Training data and labels!!!


'\nWrite Code Below\nPointers\nNeed to load the segments.txt file for segments for test videos \nOutput the CSV in correct format as shown in Evaluation Section\nId corresponds to the segments in order. \nExample - 30-150 = Id 0\n          150-428 = Id 1\n          428-575 = Id 2\nCategory is the Class of the Predicted Action\n'

In [0]:
print('Number of training videos:', len(data_feat), '; Number of training segment labels: ',len(data_labels))
print('Number of test videos: ', len(data_feat_test))

Number of training videos: 1460 ; Number of training segment labels:  1460
Number of test videos:  252


In [0]:
# First data_feat shape and number of segment labels
print('I3D feature dimension of the first video: ', data_feat[0].shape)
print('Sub-action segment labels of the first video: ', data_labels[0])

I3D feature dimension of the first video:  torch.Size([544, 400])
Sub-action segment labels of the first video:  [1, 2]


In [0]:
# Convert segment classes into list
file_class = open("/content/gdrive/My Drive/CS5242/splits/mapping_bf.txt", "r")
class_list = [line.split('\n') for line in file_class.readlines()]

file_train = open("/content/gdrive/My Drive/CS5242/training_segment.txt", "r")
train_segments = [line.split('\n') for line in file_train.readlines()]

train_segment_list = []
for i in range(len(train_segments)):
  train_segment_list.append(train_segments[i][0])

train_seg = [segment_line.split(' ') for segment_line in train_segment_list]

# Change the string format into int
train_seg_new = []
for i in train_seg:
  seg = []
  for j in i:
    seg.append(int(j))
  train_seg_new.append(seg)
print(train_seg_new[0:5])

[[9, 269, 474], [48, 376, 637, 1270, 5506, 6799, 6871], [6, 56, 218, 583, 701, 997, 1106, 1628, 1761, 2067, 2166], [23, 313, 478, 618, 743], [15, 260, 440, 714, 1276, 1340, 1866, 2067, 7475, 7551, 7930]]


In [0]:
print('Number of train videos: ',len(data_feat), '; Number of segment lists: ', len(train_seg_new))

Number of train videos:  1460 ; Number of segment lists:  1460


### 3. Continous Temporal Embedding

#### 3.1 Get 10 Activity Class

In [0]:
import os
print(sorted(os.listdir("/content/gdrive/My Drive/CS5242/groundTruth/groundTruth")))

['P16_cam01_P16_cereals.txt', 'P16_cam01_P16_friedegg.txt', 'P16_cam01_P16_juice.txt', 'P16_cam01_P16_milk.txt', 'P16_cam01_P16_pancake.txt', 'P16_cam01_P16_salat.txt', 'P16_cam01_P16_sandwich.txt', 'P16_cam01_P16_tea.txt', 'P16_stereo01_P16_cereals.txt', 'P16_stereo01_P16_coffee.txt', 'P16_stereo01_P16_friedegg.txt', 'P16_stereo01_P16_juice.txt', 'P16_stereo01_P16_milk.txt', 'P16_stereo01_P16_pancake.txt', 'P16_stereo01_P16_salat.txt', 'P16_stereo01_P16_sandwich.txt', 'P16_stereo01_P16_scrambledegg.txt', 'P16_stereo01_P16_tea.txt', 'P16_webcam01_P16_cereals.txt', 'P16_webcam01_P16_friedegg.txt', 'P16_webcam01_P16_juice.txt', 'P16_webcam01_P16_milk.txt', 'P16_webcam01_P16_pancake.txt', 'P16_webcam01_P16_salat.txt', 'P16_webcam01_P16_sandwich.txt', 'P16_webcam01_P16_scrambledegg.txt', 'P16_webcam01_P16_tea.txt', 'P16_webcam02_P16_cereals.txt', 'P16_webcam02_P16_coffee.txt', 'P16_webcam02_P16_friedegg.txt', 'P16_webcam02_P16_juice.txt', 'P16_webcam02_P16_milk.txt', 'P16_webcam02_P16_panc

In [0]:
# Get 10 Activity Class
# Make sure the sequence is correspinding to data_label list
activity_class_list = []

import os,glob,ntpath
folder_path = '/content/gdrive/My Drive/CS5242/groundTruth/groundTruth'
for filename in sorted(glob.glob(os.path.join(folder_path, '*.txt'))):
    _, activity_class_name = os.path.split(filename)
    activity_class_name = (activity_class_name.split('_')[-1]).split('.')[0]
    activity_class_list.append(activity_class_name)

print('total number of training dataset', len(activity_class_list))
print('the first 10 video activity class name')
activity_class_list[0:20]

total number of training dataset 1460
the first 10 video activity class name


['cereals',
 'friedegg',
 'juice',
 'milk',
 'pancake',
 'salat',
 'sandwich',
 'tea',
 'cereals',
 'coffee',
 'friedegg',
 'juice',
 'milk',
 'pancake',
 'salat',
 'sandwich',
 'scrambledegg',
 'tea',
 'cereals',
 'friedegg']

In [0]:
# Total number of unique activity class = 10
ten_activity_class = list(set(activity_class_list))
ten_activity_class

['coffee',
 'tea',
 'scrambledegg',
 'cereals',
 'milk',
 'sandwich',
 'salat',
 'pancake',
 'friedegg',
 'juice']

In [0]:
# Sort the activity class dataset
class_video_index = []

for name in ten_activity_class:
    class_name = name
    video_index = []
    video_index = [i for i, e in enumerate(activity_class_list) if e == name]
    class_video_index.append(video_index)

print('total number of activity class',len(class_video_index))
print('the first 1 activity class video index', class_video_index[0:1])

total number of activity class 10
the first 1 activity class video index [[9, 28, 38, 47, 55, 63, 72, 81, 85, 95, 105, 118, 137, 146, 150, 157, 166, 176, 186, 190, 196, 206, 216, 226, 233, 238, 247, 257, 265, 274, 281, 291, 301, 311, 318, 323, 333, 343, 352, 366, 376, 384, 393, 397, 407, 417, 426, 432, 452, 462, 471, 476, 484, 493, 500, 510, 519, 523, 533, 543, 548, 558, 572, 589, 597, 607, 610, 619, 638, 666, 676, 686, 696, 706, 716, 726, 736, 763, 779, 797, 807, 815, 825, 835, 845, 855, 865, 875, 885, 895, 904, 913, 923, 936, 953, 963, 973, 982, 992, 998, 1008, 1018, 1028, 1036, 1042, 1052, 1062, 1072, 1081, 1086, 1096, 1106, 1116, 1128, 1137, 1156, 1166, 1169, 1179, 1189, 1199, 1208, 1216, 1225, 1235, 1245, 1255, 1263, 1273, 1292, 1302, 1309, 1319, 1336, 1345, 1354, 1364, 1378, 1388, 1397, 1407, 1424, 1433, 1443, 1451]]


In [0]:
# For each activity class, the number of total videos under each activity class
class_video_number = []
for i in range(len(class_video_index)):
    class_video_number.append(len(class_video_index[i]))
print('number of videos for each activity class', class_video_number)

number of videos for each activity class [145, 154, 140, 157, 156, 142, 139, 135, 146, 146]


In [0]:
# Get the training data_fest for each activity class
data_feat_class_1 = []
time_stamp_class_1 = []
data_label_class_1 = []
for i in class_video_index[0]:
    data_feat_class_1.append(data_feat[i])
    time_stamp_class_1.append(np.arange(0,data_feat[i].shape[0])/(data_feat[i].shape[0]))
    data_label_class_1.append(data_labels[i])

print('total number of video in class_1:', len(data_feat_class_1))
print('total number of video with frame time stamps', len(time_stamp_class_1))
print('total number of data labels for each video in class_1', len(data_label_class_1))
print('the first video size in class_1:', data_feat_class_1[0].shape)
print('the first video size in original data_feat:', data_feat[class_video_index[0][0]].shape)
print('the first video frame time stamp size:', time_stamp_class_1[0].shape)
print('the first video frame first 20 time stamps ', time_stamp_class_1[0][0:20])
print('segment lables in the first 5 videos in class_1 excluding 0:SIL', data_label_class_1[0:10])

total number of video in class_1: 145
total number of video with frame time stamps 145
total number of data labels for each video in class_1 145
the first video size in class_1: torch.Size([571, 400])
the first video size in original data_feat: torch.Size([571, 400])
the first video frame time stamp size: (571,)
the first video frame first 20 time stamps  [0.         0.00175131 0.00350263 0.00525394 0.00700525 0.00875657
 0.01050788 0.01225919 0.01401051 0.01576182 0.01751313 0.01926445
 0.02101576 0.02276708 0.02451839 0.0262697  0.02802102 0.02977233
 0.03152364 0.03327496]
segment lables in the first 5 videos in class_1 excluding 0:SIL [[5, 2], [5, 2], [5], [5], [5], [5], [5], [5], [5], [5]]


In [0]:
# Flatten data_feat for each activity class
data_feat_class_1_flat = []
time_stamp_class_1_flat = []
data_label_class_1_flat = []
sum_video_frames = 0
for k in range(len(data_feat_class_1)):
    for q in range(len(data_feat_class_1[k])):
        data_feat_class_1_flat.append(np.array(data_feat_class_1[k][q]))
        time_stamp_class_1_flat.append(time_stamp_class_1[k][q])
    sum_video_frames += data_feat_class_1[k].shape[0]

    for p in range(len(data_label_class_1[k])):
        data_label_class_1_flat.append(data_label_class_1[k][p])

print('total numebr of video frames count for class_1', sum_video_frames)
print('total number of video frames for class_1',len(data_feat_class_1_flat))
print('total number of video frames time stamp for class_1',len(time_stamp_class_1_flat))
print('frame feature dimension', data_feat_class_1_flat[0].shape)
print('total number of segment label for all video in class_1', len(data_label_class_1_flat))

total numebr of video frames count for class_1 84560
total number of video frames for class_1 84560
total number of video frames time stamp for class_1 84560
frame feature dimension (400,)
total number of segment label for all video in class_1 380


In [0]:
data_feat_class_1_flat = np.array(data_feat_class_1_flat)
data_feat_class_1_flat = data_feat_class_1_flat.reshape(data_feat_class_1_flat.shape[0], 1, 400)
data_feat_class_1_flat.shape

(84560, 1, 400)

In [0]:
time_stamp_class_1_flat = np.array(time_stamp_class_1_flat)
time_stamp_class_1_flat = time_stamp_class_1_flat.reshape(time_stamp_class_1_flat.shape[0], 1, 1)
time_stamp_class_1_flat.shape

(84560, 1, 1)

In [0]:
# Get unique number of segment labels in activity class 1
k_class_1 =  len(set(data_label_class_1_flat))
k_class_1_name = set(data_label_class_1_flat)
print('Total unique segment labels in activity class_1', k_class_1)
print('activity class name',k_class_1_name)

Total unique segment labels in activity class_1 6
activity class name {2, 5, 6, 7, 8, 9}


#### 3.2 MLP to compute embedding features wrt time stamp

In [0]:
# Use keras to define MLP
# Use MLP to compute the embedding of features with respect to their relative time stamp
from keras.layers import Dense, Flatten, Dropout
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras import backend

backend.clear_session()
feature_dim = 400
embed_dim = 20
mlp_model = Sequential()
mlp_model.add(Dense(embed_dim * 2, input_shape = (1, feature_dim), activation='sigmoid'))
mlp_model.add(Dense(embed_dim, activation='sigmoid'))
mlp_model.add(Dense(1))
optimizer = Adam(lr=1e-3, decay=1e-4)
mlp_model.compile(loss='mse', optimizer = optimizer, metrics=['accuracy'])
print(mlp_model.summary())

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1, 40)             16040     
_________________________________________________________________
dense_2 (Dense)              (None, 1, 20)             820       
_________________________________________________________________
dense_3 (Dense)              (None, 1, 1)              21        
Total params: 16,881
Trainable params: 16,881
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# Fit model (verbose: 0=silent, 1=progress bar, 2=one line per epoch)
import time

start = time.time()

history = mlp_model.fit(data_feat_class_1_flat, time_stamp_class_1_flat, batch_size=256, validation_split=0.2, epochs=64, verbose=2, shuffle=True)

elapsed = time.time() - start
print('Computation time = ' + str(round(elapsed,2)) + 's')


Train on 67648 samples, validate on 16912 samples
Epoch 1/64
 - 2s - loss: 0.0791 - accuracy: 0.0013 - val_loss: 0.0769 - val_accuracy: 8.8694e-04
Epoch 2/64
 - 1s - loss: 0.0591 - accuracy: 0.0015 - val_loss: 0.0737 - val_accuracy: 9.4607e-04
Epoch 3/64
 - 1s - loss: 0.0469 - accuracy: 0.0016 - val_loss: 0.0818 - val_accuracy: 5.9130e-04
Epoch 4/64
 - 1s - loss: 0.0386 - accuracy: 0.0017 - val_loss: 0.0847 - val_accuracy: 7.0956e-04
Epoch 5/64
 - 1s - loss: 0.0324 - accuracy: 0.0016 - val_loss: 0.0860 - val_accuracy: 7.6868e-04
Epoch 6/64
 - 1s - loss: 0.0281 - accuracy: 0.0017 - val_loss: 0.0902 - val_accuracy: 6.5043e-04
Epoch 7/64
 - 1s - loss: 0.0252 - accuracy: 0.0017 - val_loss: 0.0902 - val_accuracy: 7.6868e-04
Epoch 8/64
 - 1s - loss: 0.0229 - accuracy: 0.0017 - val_loss: 0.0950 - val_accuracy: 5.3217e-04
Epoch 9/64
 - 1s - loss: 0.0217 - accuracy: 0.0017 - val_loss: 0.0962 - val_accuracy: 4.7304e-04
Epoch 10/64
 - 1s - loss: 0.0201 - accuracy: 0.0017 - val_loss: 0.0987 - val

In [0]:
# Extract the second last layer embedding features for KMeans with a Sequential model
get_2nd_last_layer_output = backend.function([mlp_model.layers[0].input], [mlp_model.layers[1].output])
embedding_features = get_2nd_last_layer_output([data_feat_class_1_flat])[0]
print(embedding_features.shape)

(84560, 1, 20)


In [0]:
# Save the embedding features
embedding_features_df = pd.DataFrame(embedding_features.reshape(embedding_features.shape[0], 20))

embedding_features_df.to_csv('embedding_features_mlp_coffee_class.csv', index=True)
!cp embedding_features_mlp_coffee_class.csv "/content/gdrive/My Drive/Colab Notebooks/CS5242_Grop_Proj/"

# Save the segment labels for class_1 (coffee)
data_label_class_1_flat_df = pd.DataFrame(data_label_class_1_flat)
data_label_class_1_flat_df.to_csv('segment_labels_coffee_class.csv', index=True)
!cp segment_labels_coffee_class.csv "/content/gdrive/My Drive/Colab Notebooks/CS5242_Grop_Proj/"

### References:
[1] Kukleva A, Kuehne H, Sener F, et al. Unsupervised learning of action classes with continuous temporal embedding[C]//Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition. 2019: 12066-12074.